## 1 간단한 함수 만들기

#### 제곱 함수와 n 제곱 함수 만들기

In [1]:
def my_sq(x):
    return x ** 2

def my_exp(x, n):
    return x ** n

print(my_sq(4))
print(my_exp(2, 4))

16
16


## 2 apply 메서드 사용하기 - 기초

#### 시리즈와 데이터프레임에 apply 메서드 사용하기

In [3]:
import pandas as pd

df = pd.DataFrame({'a': [10, 20, 30], 'b': [20, 30, 40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [4]:
print(df['a'] ** 2)

0    100
1    400
2    900
Name: a, dtype: int64


In [6]:
sq = df['a'].apply(my_sq)
print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


In [7]:
ex = df['a'].apply(my_exp, n=2)
print(ex)

0    100
1    400
2    900
Name: a, dtype: int64


In [8]:
ex = df['a'].apply(my_exp, n=3)
print(ex)

0     1000
1     8000
2    27000
Name: a, dtype: int64


In [9]:
def print_me(x):
    print(x)

In [10]:
print(df.apply(print_me, axis=0))

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    None
b    None
dtype: object


In [11]:
print(df['a'])
print(df['b'])

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64


In [12]:
def avg_3(x, y, z):
    return (x + y + z) / 3

In [13]:
print(df.apply(avg_3))

TypeError: ("avg_3() missing 2 required positional arguments: 'y' and 'z'", 'occurred at index a')

In [14]:
def avg_3_apply(col):
    x = col[0]
    y = col[1]
    z = col[2]
    return (x + y + z) / 3

print(df.apply(avg_3_apply))

a    20.0
b    30.0
dtype: float64


In [15]:
def avg_3_apply(col):
    sum = 0
    for item in col:
        sum += item
    return sum / df.shape[0]

In [17]:
def avg_2_apply(row):
    sum = 0
    for item in row:
        sum += item
    return sum / df.shape[1]

print(df.apply(avg_2_apply, axis=1))

0    15.0
1    25.0
2    35.0
dtype: float64


## 3 apply 메서드 사용하기 - 고급

In [18]:
import seaborn as sns

titanic = sns.load_dataset('titanic')

print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


In [19]:
import numpy as np

def count_missing(vec):
    null_vec = pd.isnull(vec)
    null_count = np.sum(null_vec)
    return null_count

In [20]:
cmis_col = titanic.apply(count_missing)
print(cmis_col)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [24]:
def prop_missing(vec):
    num = count_missing(vec)
    dem = vec.size
    return num / dem

In [25]:
pmis_col = titanic.apply(prop_missing)
print(pmis_col)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64


In [26]:
def prop_complete(vec):
    return 1 - prop_missing(vec)

In [27]:
cmis_row = titanic.apply(count_missing, axis=1)
pmis_row = titanic.apply(prop_missing, axis=1)
pcom_row = titanic.apply(prop_complete, axis=1)

print(cmis_row.head())

print(pmis_row.head())

print(pcom_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64
0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64
0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64


In [28]:
titanic['num_missing'] = titanic.apply(count_missing, axis=1)

print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [29]:
print(titanic.loc[titanic.num_missing > 1, :].sample(10))

     survived  pclass     sex  age  sibsp  parch     fare embarked   class  \
611         0       3    male  NaN      0      0   7.0500        S   Third   
778         0       3    male  NaN      0      0   7.7375        Q   Third   
36          1       3    male  NaN      0      0   7.2292        C   Third   
738         0       3    male  NaN      0      0   7.8958        S   Third   
181         0       2    male  NaN      0      0  15.0500        C  Second   
223         0       3    male  NaN      0      0   7.8958        S   Third   
766         0       1    male  NaN      0      0  39.6000        C   First   
584         0       3    male  NaN      0      0   8.7125        C   Third   
201         0       3    male  NaN      8      2  69.5500        S   Third   
863         0       3  female  NaN      8      2  69.5500        S   Third   

       who  adult_male deck  embark_town alive  alone  num_missing  
611    man        True  NaN  Southampton    no   True            2  
778